In [3]:
import os
import laspy
import numpy as np
import torch
from tqdm import tqdm
from sklearn.metrics import classification_report,confusion_matrix

# -----------------------------
# CHANGE THESE PATHS
# -----------------------------
MODEL_PATH = r"D:\lidarrrrr\anbu\dl_models\pointnet_best.pt"
RAW_FILE   = r"D:\lidarrrrr\anbu\INPUT FILE\DX3013595 PASQUILIO\LAZ\DX3013595 PASQUILIO000001.laz"
OUT_FILE   = r"d:\lidarrrrr\anbu\test\RAW_000001_PRED_PT.laz"

GT_FILE    = r"D:\lidarrrrr\anbu\OUTPUT FILE\DX3013595 PASQUILIO\LAZ\DX3013595 PASQUILIO000001.laz"   # for REAL accuracy

NPTS = 4096
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print("Device:",DEVICE)

import torch.nn as nn
import torch.nn.functional as F

class PointNet(nn.Module):
    def __init__(self,num_classes=7):
        super().__init__()

        self.conv1 = nn.Conv1d(4,64,1)
        self.conv2 = nn.Conv1d(64,128,1)
        self.conv3 = nn.Conv1d(128,256,1)
        self.conv4 = nn.Conv1d(256,512,1)

        self.fc1 = nn.Linear(512,256)
        self.fc2 = nn.Linear(256,128)
        self.fc3 = nn.Linear(128,num_classes)

    def forward(self,x):
        x = x.permute(0,2,1)

        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))

        x = torch.max(x,2)[0]

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x.unsqueeze(1).repeat(1,4096,1)
# -----------------------------
# LOAD MODEL (.pt)
# -----------------------------
ckpt = torch.load(MODEL_PATH,map_location=DEVICE)

num_cls = ckpt['num_classes']

model = PointNet(num_classes=num_cls).to(DEVICE)

model.load_state_dict(ckpt['model_state'])

model.eval()

# -----------------------------
# LOAD RAW
# -----------------------------
las = laspy.read(RAW_FILE)

xyz = np.vstack([las.x,las.y,las.z]).T.astype(np.float32)
intensity = np.array(las.intensity,dtype=np.float32)

if intensity.max()>intensity.min():
    intensity = (intensity-intensity.min())/(intensity.max()-intensity.min()+1e-6)

feat = np.concatenate([xyz,intensity[:,None]],axis=1)

# -----------------------------
# BLOCKS
# -----------------------------
def make_blocks(feat,npts=4096):
    idx = np.random.permutation(len(feat))
    blocks=[]
    for i in range(0,len(feat),npts):
        b = idx[i:i+npts]
        if len(b)<npts:
            b = np.pad(b,(0,npts-len(b)),mode='wrap')
        blocks.append(b)
    return blocks

blocks = make_blocks(feat,NPTS)

pred_lbl = np.zeros(len(feat),dtype=np.int32)

print("Predicting...")

for b in tqdm(blocks):

    xb = feat[b].copy()

    xb[:,0]-=xb[:,0].mean()
    xb[:,1]-=xb[:,1].mean()
    xb[:,2]-=xb[:,2].min()

    xt = torch.from_numpy(xb).float().unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        out = model(xt)
        p = out.argmax(2).cpu().numpy()[0]

    pred_lbl[b] = p

# -----------------------------
# SAVE PRED
# -----------------------------
out = laspy.LasData(header=las.header)
out.points = las.points
out.classification = pred_lbl.astype(np.uint8)
out.write(OUT_FILE)

print("Saved:",OUT_FILE)

# -----------------------------
# REAL ACCURACY (GT vs PRED)
# -----------------------------
gt = laspy.read(GT_FILE)

y_true = np.array(gt.classification)
y_pred = pred_lbl

mask = np.isin(y_true,[1,2,3,6])

y_true = y_true[mask]
y_pred = y_pred[mask]

rep = classification_report(y_true,y_pred,labels=[1,2,3,6],output_dict=True,zero_division=0)
cm  = confusion_matrix(y_true,y_pred,labels=[1,2,3,6])

print("\nREAL ACCURACY\n")

for c in [2,3,6]:
    acc = rep[str(c)]['recall']*100
    print(f"Class {c} Accuracy: {acc:.2f}%")

overall = np.trace(cm)/np.sum(cm)*100
print(f"\nOverall Accuracy: {overall:.2f}%")

Device: cuda


RuntimeError: Error(s) in loading state_dict for PointNet:
	Missing key(s) in state_dict: "conv1.weight", "conv1.bias", "conv2.weight", "conv2.bias", "conv3.weight", "conv3.bias", "conv4.weight", "conv4.bias", "fc3.weight", "fc3.bias". 
	Unexpected key(s) in state_dict: "mlp1.weight", "mlp1.bias", "mlp2.weight", "mlp2.bias", "mlp3.weight", "mlp3.bias". 
	size mismatch for fc1.weight: copying a param with shape torch.Size([128, 256, 1]) from checkpoint, the shape in current model is torch.Size([256, 512]).
	size mismatch for fc1.bias: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([256]).
	size mismatch for fc2.weight: copying a param with shape torch.Size([5, 128, 1]) from checkpoint, the shape in current model is torch.Size([128, 256]).
	size mismatch for fc2.bias: copying a param with shape torch.Size([5]) from checkpoint, the shape in current model is torch.Size([128]).

In [2]:
ckpt = torch.load(MODEL_PATH,map_location="cpu")

print(type(ckpt))
print(ckpt.keys() if isinstance(ckpt,dict) else "Not a dict")

<class 'dict'>
dict_keys(['model_state', 'num_classes', 'class_weights'])


In [5]:
ckpt = torch.load(MODEL_PATH, map_location="cpu")

for k in ckpt['model_state'].keys():
    print(k)

mlp1.weight
mlp1.bias
mlp2.weight
mlp2.bias
mlp3.weight
mlp3.bias
fc1.weight
fc1.bias
fc2.weight
fc2.bias


In [6]:
import os
import numpy as np
import laspy
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

# -----------------------------
# EDIT THESE PATHS
# -----------------------------
MODEL_PATH = r"D:\lidarrrrr\anbu\dl_models\pointnet_best.pt"   # your .pt
GT_FILE    = r"D:\lidarrrrr\anbu\OUTPUT FILE\DX3013595 PASQUILIO\LAZ\DX3013595 PASQUILIO000001.laz"          # classified (ground truth)
RAW_FILE   = r"D:\lidarrrrr\anbu\INPUT FILE\DX3013595 PASQUILIO\LAZ\DX3013595 PASQUILIO000001.laz"         # will be created from GT if not exists
PRED_FILE  = r"d:\lidarrrrr\anbu\test\RAW_000001_PRED_PT.laz"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

# -----------------------------
# 1) MAKE RAW FROM GT (same points, perfect for real accuracy)
# -----------------------------
def make_raw_from_gt(gt_path, raw_path):
    os.makedirs(os.path.dirname(raw_path), exist_ok=True)
    las = laspy.read(gt_path)
    print("GT classes:", np.unique(las.classification))
    las.classification[:] = 1
    las.write(raw_path)
    print("✅ RAW created:", raw_path)

if not os.path.exists(RAW_FILE):
    make_raw_from_gt(GT_FILE, RAW_FILE)

# -----------------------------
# 2) BUILD MODEL FROM CHECKPOINT SHAPES (NO GUESSING)
# -----------------------------
class PointNetFromCkpt(nn.Module):
    def __init__(self, sd):
        super().__init__()
        # Conv1d weights are [out_channels, in_channels, 1]
        def conv_from(key):
            w = sd[key + ".weight"]
            out_ch, in_ch, k = w.shape
            assert k == 1, f"{key} kernel is not 1, got {k}"
            return nn.Conv1d(in_ch, out_ch, kernel_size=1, bias=True)

        self.mlp1 = conv_from("mlp1")
        self.mlp2 = conv_from("mlp2")
        self.mlp3 = conv_from("mlp3")
        self.fc1  = conv_from("fc1")
        self.fc2  = conv_from("fc2")

    def forward(self, x):
        # x: [B, N, C] -> [B, C, N]
        x = x.permute(0, 2, 1)
        x = F.relu(self.mlp1(x))
        x = F.relu(self.mlp2(x))
        x = F.relu(self.mlp3(x))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)          # [B, num_classes, N]
        return x.permute(0, 2, 1) # [B, N, num_classes]

ckpt = torch.load(MODEL_PATH, map_location="cpu")

# Your checkpoint keys are: model_state, num_classes, class_weights
sd = ckpt["model_state"]
num_classes = int(ckpt.get("num_classes", sd["fc2.weight"].shape[0]))
print("Checkpoint num_classes:", num_classes)

model = PointNetFromCkpt(sd).to(DEVICE)
missing, unexpected = model.load_state_dict(sd, strict=False)
print("✅ Loaded PointNet weights.")
if missing:    print("Missing:", missing)
if unexpected: print("Unexpected:", unexpected)
model.eval()

# -----------------------------
# 3) FEATURES (match your training: XYZ + Intensity + returns)
#    If your model was trained with a different C, we auto-pad/trim.
# -----------------------------
def build_features(las):
    dims = set(las.point_format.dimension_names)
    xyz = np.vstack([las.x, las.y, las.z]).T.astype(np.float32)

    intensity = np.array(las.intensity, dtype=np.float32) if "intensity" in dims else np.zeros(len(xyz), np.float32)
    rn = np.array(las.return_number, dtype=np.float32) if "return_number" in dims else np.ones(len(xyz), np.float32)
    nr = np.array(las.number_of_returns, dtype=np.float32) if "number_of_returns" in dims else np.ones(len(xyz), np.float32)

    if intensity.max() > intensity.min():
        intensity = (intensity - intensity.min()) / (intensity.max() - intensity.min() + 1e-6)
    ret_ratio = rn / (nr + 1e-6)

    # base features
    X = np.column_stack([xyz, intensity, ret_ratio, nr]).astype(np.float32)  # C=6
    return X

def fit_C(X, target_C):
    C = X.shape[1]
    if C == target_C:
        return X
    if C > target_C:
        return X[:, :target_C]
    # pad with zeros
    pad = np.zeros((X.shape[0], target_C - C), dtype=X.dtype)
    return np.concatenate([X, pad], axis=1)

# model input channels = mlp1 in_channels
C_in = model.mlp1.in_channels
print("Model expects input features C =", C_in)

# -----------------------------
# 4) PREDICT (block by block)
# -----------------------------
def make_blocks(n_points, npts=4096):
    idx = np.arange(n_points)
    np.random.shuffle(idx)
    blocks = []
    for i in range(0, n_points, npts):
        b = idx[i:i+npts]
        if len(b) < npts:
            b = np.pad(b, (0, npts-len(b)), mode="wrap")
        blocks.append(b)
    return blocks

raw = laspy.read(RAW_FILE)
X_all = build_features(raw)
X_all = fit_C(X_all, C_in)

N = X_all.shape[0]
NPTS = 4096
blocks = make_blocks(N, NPTS)

pred = np.zeros(N, dtype=np.int32)

print("Predicting blocks:", len(blocks))
with torch.no_grad():
    for b in tqdm(blocks):
        xb = X_all[b].copy()

        # simple per-block normalization
        xb[:, 0] -= xb[:, 0].mean()
        xb[:, 1] -= xb[:, 1].mean()
        xb[:, 2] -= xb[:, 2].min()

        xt = torch.from_numpy(xb).float().unsqueeze(0).to(DEVICE)  # [1,NPTS,C]
        logits = model(xt)                                         # [1,NPTS,num_classes]
        pb = logits.argmax(-1).squeeze(0).cpu().numpy().astype(np.int32)
        pred[b] = pb

# Save predicted LAZ
out = laspy.LasData(header=raw.header)
out.points = raw.points
out.classification = pred.astype(np.uint8)
out.write(PRED_FILE)
print("✅ Saved:", PRED_FILE)

# -----------------------------
# 5) REAL ACCURACY (GT vs PRED) — SAME POINTS = TRUE ACCURACY
# -----------------------------
gt = laspy.read(GT_FILE)
y_true = np.array(gt.classification, dtype=np.int32)
y_pred = pred.astype(np.int32)

# report only main classes (edit if you want more)
labels = [1, 2, 3, 6]
mask = np.isin(y_true, labels)

rep = classification_report(y_true[mask], y_pred[mask], labels=labels, digits=4, zero_division=0, output_dict=True)
cm  = confusion_matrix(y_true[mask], y_pred[mask], labels=labels)
overall = (np.trace(cm) / np.sum(cm)) * 100.0 if cm.sum() > 0 else 0.0

print("\nREAL ACCURACY (PointNet)\n")
for c in [2, 3, 6]:
    print(f"Class {c} Accuracy (Recall): {rep[str(c)]['recall']*100:.2f}%")
print(f"\nOverall Accuracy: {overall:.2f}%")

Device: cuda
GPU: NVIDIA GeForce RTX 3050
Checkpoint num_classes: 5
✅ Loaded PointNet weights.
Model expects input features C = 10
Predicting blocks: 371


100%|██████████| 371/371 [00:05<00:00, 64.10it/s]


✅ Saved: d:\lidarrrrr\anbu\test\RAW_000001_PRED_PT.laz

REAL ACCURACY (PointNet)

Class 2 Accuracy (Recall): 99.99%
Class 3 Accuracy (Recall): 0.00%
Class 6 Accuracy (Recall): 0.00%

Overall Accuracy: 27.51%


In [22]:
import os
import numpy as np
import laspy
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

# -----------------------------
# EDIT THESE PATHS
# -----------------------------
MODEL_PATH = r"D:\lidarrrrr\anbu\dl_models\pointnet_best.pt"   # your .pt
GT_FILE    = r"D:\lidarrrrr\anbu\OUTPUT FILE\DX3013595 PASQUILIO\LAZ\DX3013595 PASQUILIO000002.laz"          # classified (ground truth)
RAW_FILE   = r"D:\lidarrrrr\anbu\INPUT FILE\DX3013595 PASQUILIO\LAZ\DX3013595 PASQUILIO000002.laz"         # will be created from GT if not exists
PRED_FILE  = r"d:\lidarrrrr\anbu\test\RAW_000002_PRED_PT.laz"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

# -----------------------------
# 1) MAKE RAW FROM GT (same points, perfect for real accuracy)
# -----------------------------
def make_raw_from_gt(gt_path, raw_path):
    os.makedirs(os.path.dirname(raw_path), exist_ok=True)
    las = laspy.read(gt_path)
    print("GT classes:", np.unique(las.classification))
    las.classification[:] = 1
    las.write(raw_path)
    print("✅ RAW created:", raw_path)

if not os.path.exists(RAW_FILE):
    make_raw_from_gt(GT_FILE, RAW_FILE)

# -----------------------------
# 2) BUILD MODEL FROM CHECKPOINT SHAPES (NO GUESSING)
# -----------------------------
class PointNetFromCkpt(nn.Module):
    def __init__(self, sd):
        super().__init__()
        # Conv1d weights are [out_channels, in_channels, 1]
        def conv_from(key):
            w = sd[key + ".weight"]
            out_ch, in_ch, k = w.shape
            assert k == 1, f"{key} kernel is not 1, got {k}"
            return nn.Conv1d(in_ch, out_ch, kernel_size=1, bias=True)

        self.mlp1 = conv_from("mlp1")
        self.mlp2 = conv_from("mlp2")
        self.mlp3 = conv_from("mlp3")
        self.fc1  = conv_from("fc1")
        self.fc2  = conv_from("fc2")

    def forward(self, x):
        # x: [B, N, C] -> [B, C, N]
        x = x.permute(0, 2, 1)
        x = F.relu(self.mlp1(x))
        x = F.relu(self.mlp2(x))
        x = F.relu(self.mlp3(x))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)          # [B, num_classes, N]
        return x.permute(0, 2, 1) # [B, N, num_classes]

ckpt = torch.load(MODEL_PATH, map_location="cpu")

# Your checkpoint keys are: model_state, num_classes, class_weights
sd = ckpt["model_state"]
num_classes = int(ckpt.get("num_classes", sd["fc2.weight"].shape[0]))
print("Checkpoint num_classes:", num_classes)

model = PointNetFromCkpt(sd).to(DEVICE)
missing, unexpected = model.load_state_dict(sd, strict=False)
print("✅ Loaded PointNet weights.")
if missing:    print("Missing:", missing)
if unexpected: print("Unexpected:", unexpected)
model.eval()

# -----------------------------
# 3) FEATURES (match your training: XYZ + Intensity + returns)
#    If your model was trained with a different C, we auto-pad/trim.
# -----------------------------
def build_features(las):
    dims = set(las.point_format.dimension_names)
    xyz = np.vstack([las.x, las.y, las.z]).T.astype(np.float32)

    intensity = np.array(las.intensity, dtype=np.float32) if "intensity" in dims else np.zeros(len(xyz), np.float32)
    rn = np.array(las.return_number, dtype=np.float32) if "return_number" in dims else np.ones(len(xyz), np.float32)
    nr = np.array(las.number_of_returns, dtype=np.float32) if "number_of_returns" in dims else np.ones(len(xyz), np.float32)

    if intensity.max() > intensity.min():
        intensity = (intensity - intensity.min()) / (intensity.max() - intensity.min() + 1e-6)
    ret_ratio = rn / (nr + 1e-6)

    # base features
    X = np.column_stack([xyz, intensity, ret_ratio, nr]).astype(np.float32)  # C=6
    return X

def fit_C(X, target_C):
    C = X.shape[1]
    if C == target_C:
        return X
    if C > target_C:
        return X[:, :target_C]
    # pad with zeros
    pad = np.zeros((X.shape[0], target_C - C), dtype=X.dtype)
    return np.concatenate([X, pad], axis=1)

# model input channels = mlp1 in_channels
C_in = model.mlp1.in_channels
print("Model expects input features C =", C_in)

# -----------------------------
# 4) PREDICT (block by block)
# -----------------------------
def make_blocks(n_points, npts=4096):
    idx = np.arange(n_points)
    np.random.shuffle(idx)
    blocks = []
    for i in range(0, n_points, npts):
        b = idx[i:i+npts]
        if len(b) < npts:
            b = np.pad(b, (0, npts-len(b)), mode="wrap")
        blocks.append(b)
    return blocks

raw = laspy.read(RAW_FILE)
X_all = build_features(raw)
X_all = fit_C(X_all, C_in)

N = X_all.shape[0]
NPTS = 4096
blocks = make_blocks(N, NPTS)

pred = np.zeros(N, dtype=np.int32)

print("Predicting blocks:", len(blocks))
with torch.no_grad():
    for b in tqdm(blocks):
        xb = X_all[b].copy()

        # simple per-block normalization
        xb[:, 0] -= xb[:, 0].mean()
        xb[:, 1] -= xb[:, 1].mean()
        xb[:, 2] -= xb[:, 2].min()

        xt = torch.from_numpy(xb).float().unsqueeze(0).to(DEVICE)  # [1,NPTS,C]
        logits = model(xt)                                         # [1,NPTS,num_classes]
        pb = logits.argmax(-1).squeeze(0).cpu().numpy().astype(np.int32)
        pred[b] = pb

# Save predicted LAZ
out = laspy.LasData(header=raw.header)
out.points = raw.points
out.classification = pred.astype(np.uint8)
out.write(PRED_FILE)
print("✅ Saved:", PRED_FILE)

# -----------------------------
# 5) REAL ACCURACY (GT vs PRED) — SAME POINTS = TRUE ACCURACY
# -----------------------------
gt = laspy.read(GT_FILE)
y_true = np.array(gt.classification, dtype=np.int32)
y_pred = pred.astype(np.int32)

# report only main classes (edit if you want more)
labels = [1, 2, 3, 6]
mask = np.isin(y_true, labels)

rep = classification_report(y_true[mask], y_pred[mask], labels=labels, digits=4, zero_division=0, output_dict=True)
cm  = confusion_matrix(y_true[mask], y_pred[mask], labels=labels)
overall = (np.trace(cm) / np.sum(cm)) * 100.0 if cm.sum() > 0 else 0.0

print("\nREAL ACCURACY (PointNet)\n")
for c in [2, 3, 6]:
    print(f"Class {c} Accuracy (Recall): {rep[str(c)]['recall']*100:.2f}%")
print(f"\nOverall Accuracy: {overall:.2f}%")

Device: cuda
GPU: NVIDIA GeForce RTX 3050
Checkpoint num_classes: 5
✅ Loaded PointNet weights.
Model expects input features C = 10
Predicting blocks: 853


100%|██████████| 853/853 [00:01<00:00, 517.86it/s]


✅ Saved: d:\lidarrrrr\anbu\test\RAW_000002_PRED_PT.laz

REAL ACCURACY (PointNet)

Class 2 Accuracy (Recall): 99.99%
Class 3 Accuracy (Recall): 0.01%
Class 6 Accuracy (Recall): 0.00%

Overall Accuracy: 28.87%


In [24]:
import os
import numpy as np
import laspy
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

# -----------------------------
# EDIT THESE PATHS
# -----------------------------
MODEL_PATH = r"D:\lidarrrrr\anbu\dl_models\pointnet_best.pt"   # your .pt
GT_FILE    = r"D:\lidarrrrr\anbu\OUTPUT FILE\DX3013595 PASQUILIO\LAZ\DX3013595 PASQUILIO000003.laz"          # classified (ground truth)
RAW_FILE   = r"D:\lidarrrrr\anbu\INPUT FILE\DX3013595 PASQUILIO\LAZ\DX3013595 PASQUILIO000003.laz"         # will be created from GT if not exists
PRED_FILE  = r"d:\lidarrrrr\anbu\test\RAW_000003_PRED_PT.laz"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

# -----------------------------
# 1) MAKE RAW FROM GT (same points, perfect for real accuracy)
# -----------------------------
def make_raw_from_gt(gt_path, raw_path):
    os.makedirs(os.path.dirname(raw_path), exist_ok=True)
    las = laspy.read(gt_path)
    print("GT classes:", np.unique(las.classification))
    las.classification[:] = 1
    las.write(raw_path)
    print("✅ RAW created:", raw_path)

if not os.path.exists(RAW_FILE):
    make_raw_from_gt(GT_FILE, RAW_FILE)

# -----------------------------
# 2) BUILD MODEL FROM CHECKPOINT SHAPES (NO GUESSING)
# -----------------------------
class PointNetFromCkpt(nn.Module):
    def __init__(self, sd):
        super().__init__()
        # Conv1d weights are [out_channels, in_channels, 1]
        def conv_from(key):
            w = sd[key + ".weight"]
            out_ch, in_ch, k = w.shape
            assert k == 1, f"{key} kernel is not 1, got {k}"
            return nn.Conv1d(in_ch, out_ch, kernel_size=1, bias=True)

        self.mlp1 = conv_from("mlp1")
        self.mlp2 = conv_from("mlp2")
        self.mlp3 = conv_from("mlp3")
        self.fc1  = conv_from("fc1")
        self.fc2  = conv_from("fc2")

    def forward(self, x):
        # x: [B, N, C] -> [B, C, N]
        x = x.permute(0, 2, 1)
        x = F.relu(self.mlp1(x))
        x = F.relu(self.mlp2(x))
        x = F.relu(self.mlp3(x))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)          # [B, num_classes, N]
        return x.permute(0, 2, 1) # [B, N, num_classes]

ckpt = torch.load(MODEL_PATH, map_location="cpu")

# Your checkpoint keys are: model_state, num_classes, class_weights
sd = ckpt["model_state"]
num_classes = int(ckpt.get("num_classes", sd["fc2.weight"].shape[0]))
print("Checkpoint num_classes:", num_classes)

model = PointNetFromCkpt(sd).to(DEVICE)
missing, unexpected = model.load_state_dict(sd, strict=False)
print("✅ Loaded PointNet weights.")
if missing:    print("Missing:", missing)
if unexpected: print("Unexpected:", unexpected)
model.eval()

# -----------------------------
# 3) FEATURES (match your training: XYZ + Intensity + returns)
#    If your model was trained with a different C, we auto-pad/trim.
# -----------------------------
def build_features(las):
    dims = set(las.point_format.dimension_names)
    xyz = np.vstack([las.x, las.y, las.z]).T.astype(np.float32)

    intensity = np.array(las.intensity, dtype=np.float32) if "intensity" in dims else np.zeros(len(xyz), np.float32)
    rn = np.array(las.return_number, dtype=np.float32) if "return_number" in dims else np.ones(len(xyz), np.float32)
    nr = np.array(las.number_of_returns, dtype=np.float32) if "number_of_returns" in dims else np.ones(len(xyz), np.float32)

    if intensity.max() > intensity.min():
        intensity = (intensity - intensity.min()) / (intensity.max() - intensity.min() + 1e-6)
    ret_ratio = rn / (nr + 1e-6)

    # base features
    X = np.column_stack([xyz, intensity, ret_ratio, nr]).astype(np.float32)  # C=6
    return X

def fit_C(X, target_C):
    C = X.shape[1]
    if C == target_C:
        return X
    if C > target_C:
        return X[:, :target_C]
    # pad with zeros
    pad = np.zeros((X.shape[0], target_C - C), dtype=X.dtype)
    return np.concatenate([X, pad], axis=1)

# model input channels = mlp1 in_channels
C_in = model.mlp1.in_channels
print("Model expects input features C =", C_in)

# -----------------------------
# 4) PREDICT (block by block)
# -----------------------------
def make_blocks(n_points, npts=4096):
    idx = np.arange(n_points)
    np.random.shuffle(idx)
    blocks = []
    for i in range(0, n_points, npts):
        b = idx[i:i+npts]
        if len(b) < npts:
            b = np.pad(b, (0, npts-len(b)), mode="wrap")
        blocks.append(b)
    return blocks

raw = laspy.read(RAW_FILE)
X_all = build_features(raw)
X_all = fit_C(X_all, C_in)

N = X_all.shape[0]
NPTS = 4096
blocks = make_blocks(N, NPTS)

pred = np.zeros(N, dtype=np.int32)

print("Predicting blocks:", len(blocks))
with torch.no_grad():
    for b in tqdm(blocks):
        xb = X_all[b].copy()

        # simple per-block normalization
        xb[:, 0] -= xb[:, 0].mean()
        xb[:, 1] -= xb[:, 1].mean()
        xb[:, 2] -= xb[:, 2].min()

        xt = torch.from_numpy(xb).float().unsqueeze(0).to(DEVICE)  # [1,NPTS,C]
        logits = model(xt)                                         # [1,NPTS,num_classes]
        pb = logits.argmax(-1).squeeze(0).cpu().numpy().astype(np.int32)
        pred[b] = pb

# Save predicted LAZ
out = laspy.LasData(header=raw.header)
out.points = raw.points
out.classification = pred.astype(np.uint8)
out.write(PRED_FILE)
print("✅ Saved:", PRED_FILE)

# -----------------------------
# 5) REAL ACCURACY (GT vs PRED) — SAME POINTS = TRUE ACCURACY
# -----------------------------
gt = laspy.read(GT_FILE)
y_true = np.array(gt.classification, dtype=np.int32)
y_pred = pred.astype(np.int32)

# report only main classes (edit if you want more)
labels = [1, 2, 3, 6]
mask = np.isin(y_true, labels)

rep = classification_report(y_true[mask], y_pred[mask], labels=labels, digits=4, zero_division=0, output_dict=True)
cm  = confusion_matrix(y_true[mask], y_pred[mask], labels=labels)
overall = (np.trace(cm) / np.sum(cm)) * 100.0 if cm.sum() > 0 else 0.0

print("\nREAL ACCURACY (PointNet)\n")
for c in [2, 3, 6]:
    print(f"Class {c} Accuracy (Recall): {rep[str(c)]['recall']*100:.2f}%")
print(f"\nOverall Accuracy: {overall:.2f}%")

Device: cuda
GPU: NVIDIA GeForce RTX 3050
Checkpoint num_classes: 5
✅ Loaded PointNet weights.
Model expects input features C = 10
Predicting blocks: 552


100%|██████████| 552/552 [00:01<00:00, 548.14it/s]


✅ Saved: d:\lidarrrrr\anbu\test\RAW_000003_PRED_PT.laz

REAL ACCURACY (PointNet)

Class 2 Accuracy (Recall): 99.89%
Class 3 Accuracy (Recall): 0.33%
Class 6 Accuracy (Recall): 0.00%

Overall Accuracy: 49.42%


In [25]:
import os
import numpy as np
import laspy
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

# -----------------------------
# EDIT THESE PATHS
# -----------------------------
MODEL_PATH = r"D:\lidarrrrr\anbu\dl_models\pointnet_best.pt"   # your .pt
GT_FILE    = r"D:\lidarrrrr\anbu\OUTPUT FILE\DX3013595 PASQUILIO\LAZ\DX3013595 PASQUILIO000004.laz"          # classified (ground truth)
RAW_FILE   = r"D:\lidarrrrr\anbu\INPUT FILE\DX3013595 PASQUILIO\LAZ\DX3013595 PASQUILIO000004.laz"         # will be created from GT if not exists
PRED_FILE  = r"d:\lidarrrrr\anbu\test\RAW_000004_PRED_PT.laz"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

# -----------------------------
# 1) MAKE RAW FROM GT (same points, perfect for real accuracy)
# -----------------------------
def make_raw_from_gt(gt_path, raw_path):
    os.makedirs(os.path.dirname(raw_path), exist_ok=True)
    las = laspy.read(gt_path)
    print("GT classes:", np.unique(las.classification))
    las.classification[:] = 1
    las.write(raw_path)
    print("✅ RAW created:", raw_path)

if not os.path.exists(RAW_FILE):
    make_raw_from_gt(GT_FILE, RAW_FILE)

# -----------------------------
# 2) BUILD MODEL FROM CHECKPOINT SHAPES (NO GUESSING)
# -----------------------------
class PointNetFromCkpt(nn.Module):
    def __init__(self, sd):
        super().__init__()
        # Conv1d weights are [out_channels, in_channels, 1]
        def conv_from(key):
            w = sd[key + ".weight"]
            out_ch, in_ch, k = w.shape
            assert k == 1, f"{key} kernel is not 1, got {k}"
            return nn.Conv1d(in_ch, out_ch, kernel_size=1, bias=True)

        self.mlp1 = conv_from("mlp1")
        self.mlp2 = conv_from("mlp2")
        self.mlp3 = conv_from("mlp3")
        self.fc1  = conv_from("fc1")
        self.fc2  = conv_from("fc2")

    def forward(self, x):
        # x: [B, N, C] -> [B, C, N]
        x = x.permute(0, 2, 1)
        x = F.relu(self.mlp1(x))
        x = F.relu(self.mlp2(x))
        x = F.relu(self.mlp3(x))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)          # [B, num_classes, N]
        return x.permute(0, 2, 1) # [B, N, num_classes]

ckpt = torch.load(MODEL_PATH, map_location="cpu")

# Your checkpoint keys are: model_state, num_classes, class_weights
sd = ckpt["model_state"]
num_classes = int(ckpt.get("num_classes", sd["fc2.weight"].shape[0]))
print("Checkpoint num_classes:", num_classes)

model = PointNetFromCkpt(sd).to(DEVICE)
missing, unexpected = model.load_state_dict(sd, strict=False)
print("✅ Loaded PointNet weights.")
if missing:    print("Missing:", missing)
if unexpected: print("Unexpected:", unexpected)
model.eval()

# -----------------------------
# 3) FEATURES (match your training: XYZ + Intensity + returns)
#    If your model was trained with a different C, we auto-pad/trim.
# -----------------------------
def build_features(las):
    dims = set(las.point_format.dimension_names)
    xyz = np.vstack([las.x, las.y, las.z]).T.astype(np.float32)

    intensity = np.array(las.intensity, dtype=np.float32) if "intensity" in dims else np.zeros(len(xyz), np.float32)
    rn = np.array(las.return_number, dtype=np.float32) if "return_number" in dims else np.ones(len(xyz), np.float32)
    nr = np.array(las.number_of_returns, dtype=np.float32) if "number_of_returns" in dims else np.ones(len(xyz), np.float32)

    if intensity.max() > intensity.min():
        intensity = (intensity - intensity.min()) / (intensity.max() - intensity.min() + 1e-6)
    ret_ratio = rn / (nr + 1e-6)

    # base features
    X = np.column_stack([xyz, intensity, ret_ratio, nr]).astype(np.float32)  # C=6
    return X

def fit_C(X, target_C):
    C = X.shape[1]
    if C == target_C:
        return X
    if C > target_C:
        return X[:, :target_C]
    # pad with zeros
    pad = np.zeros((X.shape[0], target_C - C), dtype=X.dtype)
    return np.concatenate([X, pad], axis=1)

# model input channels = mlp1 in_channels
C_in = model.mlp1.in_channels
print("Model expects input features C =", C_in)

# -----------------------------
# 4) PREDICT (block by block)
# -----------------------------
def make_blocks(n_points, npts=4096):
    idx = np.arange(n_points)
    np.random.shuffle(idx)
    blocks = []
    for i in range(0, n_points, npts):
        b = idx[i:i+npts]
        if len(b) < npts:
            b = np.pad(b, (0, npts-len(b)), mode="wrap")
        blocks.append(b)
    return blocks

raw = laspy.read(RAW_FILE)
X_all = build_features(raw)
X_all = fit_C(X_all, C_in)

N = X_all.shape[0]
NPTS = 4096
blocks = make_blocks(N, NPTS)

pred = np.zeros(N, dtype=np.int32)

print("Predicting blocks:", len(blocks))
with torch.no_grad():
    for b in tqdm(blocks):
        xb = X_all[b].copy()

        # simple per-block normalization
        xb[:, 0] -= xb[:, 0].mean()
        xb[:, 1] -= xb[:, 1].mean()
        xb[:, 2] -= xb[:, 2].min()

        xt = torch.from_numpy(xb).float().unsqueeze(0).to(DEVICE)  # [1,NPTS,C]
        logits = model(xt)                                         # [1,NPTS,num_classes]
        pb = logits.argmax(-1).squeeze(0).cpu().numpy().astype(np.int32)
        pred[b] = pb

# Save predicted LAZ
out = laspy.LasData(header=raw.header)
out.points = raw.points
out.classification = pred.astype(np.uint8)
out.write(PRED_FILE)
print("✅ Saved:", PRED_FILE)

# -----------------------------
# 5) REAL ACCURACY (GT vs PRED) — SAME POINTS = TRUE ACCURACY
# -----------------------------
gt = laspy.read(GT_FILE)
y_true = np.array(gt.classification, dtype=np.int32)
y_pred = pred.astype(np.int32)

# report only main classes (edit if you want more)
labels = [1, 2, 3, 6]
mask = np.isin(y_true, labels)

rep = classification_report(y_true[mask], y_pred[mask], labels=labels, digits=4, zero_division=0, output_dict=True)
cm  = confusion_matrix(y_true[mask], y_pred[mask], labels=labels)
overall = (np.trace(cm) / np.sum(cm)) * 100.0 if cm.sum() > 0 else 0.0

print("\nREAL ACCURACY (PointNet)\n")
for c in [2, 3, 6]:
    print(f"Class {c} Accuracy (Recall): {rep[str(c)]['recall']*100:.2f}%")
print(f"\nOverall Accuracy: {overall:.2f}%")

Device: cuda
GPU: NVIDIA GeForce RTX 3050
Checkpoint num_classes: 5
✅ Loaded PointNet weights.
Model expects input features C = 10
Predicting blocks: 45


100%|██████████| 45/45 [00:00<00:00, 144.44it/s]

✅ Saved: d:\lidarrrrr\anbu\test\RAW_000004_PRED_PT.laz

REAL ACCURACY (PointNet)

Class 2 Accuracy (Recall): 100.00%
Class 3 Accuracy (Recall): 0.00%
Class 6 Accuracy (Recall): 0.00%

Overall Accuracy: 32.36%


In [26]:
import os
import numpy as np
import laspy
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

# -----------------------------
# EDIT THESE PATHS
# -----------------------------
MODEL_PATH = r"D:\lidarrrrr\anbu\dl_models\pointnet_best.pt"   # your .pt
GT_FILE    = r"D:\lidarrrrr\anbu\OUTPUT FILE\DX3013595 PASQUILIO\LAZ\DX3013595 PASQUILIO000005.laz"          # classified (ground truth)
RAW_FILE   = r"D:\lidarrrrr\anbu\INPUT FILE\DX3013595 PASQUILIO\LAZ\DX3013595 PASQUILIO000005.laz"         # will be created from GT if not exists
PRED_FILE  = r"d:\lidarrrrr\anbu\test\RAW_000005_PRED_PT.laz"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

# -----------------------------
# 1) MAKE RAW FROM GT (same points, perfect for real accuracy)
# -----------------------------
def make_raw_from_gt(gt_path, raw_path):
    os.makedirs(os.path.dirname(raw_path), exist_ok=True)
    las = laspy.read(gt_path)
    print("GT classes:", np.unique(las.classification))
    las.classification[:] = 1
    las.write(raw_path)
    print("✅ RAW created:", raw_path)

if not os.path.exists(RAW_FILE):
    make_raw_from_gt(GT_FILE, RAW_FILE)

# -----------------------------
# 2) BUILD MODEL FROM CHECKPOINT SHAPES (NO GUESSING)
# -----------------------------
class PointNetFromCkpt(nn.Module):
    def __init__(self, sd):
        super().__init__()
        # Conv1d weights are [out_channels, in_channels, 1]
        def conv_from(key):
            w = sd[key + ".weight"]
            out_ch, in_ch, k = w.shape
            assert k == 1, f"{key} kernel is not 1, got {k}"
            return nn.Conv1d(in_ch, out_ch, kernel_size=1, bias=True)

        self.mlp1 = conv_from("mlp1")
        self.mlp2 = conv_from("mlp2")
        self.mlp3 = conv_from("mlp3")
        self.fc1  = conv_from("fc1")
        self.fc2  = conv_from("fc2")

    def forward(self, x):
        # x: [B, N, C] -> [B, C, N]
        x = x.permute(0, 2, 1)
        x = F.relu(self.mlp1(x))
        x = F.relu(self.mlp2(x))
        x = F.relu(self.mlp3(x))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)          # [B, num_classes, N]
        return x.permute(0, 2, 1) # [B, N, num_classes]

ckpt = torch.load(MODEL_PATH, map_location="cpu")

# Your checkpoint keys are: model_state, num_classes, class_weights
sd = ckpt["model_state"]
num_classes = int(ckpt.get("num_classes", sd["fc2.weight"].shape[0]))
print("Checkpoint num_classes:", num_classes)

model = PointNetFromCkpt(sd).to(DEVICE)
missing, unexpected = model.load_state_dict(sd, strict=False)
print("✅ Loaded PointNet weights.")
if missing:    print("Missing:", missing)
if unexpected: print("Unexpected:", unexpected)
model.eval()

# -----------------------------
# 3) FEATURES (match your training: XYZ + Intensity + returns)
#    If your model was trained with a different C, we auto-pad/trim.
# -----------------------------
def build_features(las):
    dims = set(las.point_format.dimension_names)
    xyz = np.vstack([las.x, las.y, las.z]).T.astype(np.float32)

    intensity = np.array(las.intensity, dtype=np.float32) if "intensity" in dims else np.zeros(len(xyz), np.float32)
    rn = np.array(las.return_number, dtype=np.float32) if "return_number" in dims else np.ones(len(xyz), np.float32)
    nr = np.array(las.number_of_returns, dtype=np.float32) if "number_of_returns" in dims else np.ones(len(xyz), np.float32)

    if intensity.max() > intensity.min():
        intensity = (intensity - intensity.min()) / (intensity.max() - intensity.min() + 1e-6)
    ret_ratio = rn / (nr + 1e-6)

    # base features
    X = np.column_stack([xyz, intensity, ret_ratio, nr]).astype(np.float32)  # C=6
    return X

def fit_C(X, target_C):
    C = X.shape[1]
    if C == target_C:
        return X
    if C > target_C:
        return X[:, :target_C]
    # pad with zeros
    pad = np.zeros((X.shape[0], target_C - C), dtype=X.dtype)
    return np.concatenate([X, pad], axis=1)

# model input channels = mlp1 in_channels
C_in = model.mlp1.in_channels
print("Model expects input features C =", C_in)

# -----------------------------
# 4) PREDICT (block by block)
# -----------------------------
def make_blocks(n_points, npts=4096):
    idx = np.arange(n_points)
    np.random.shuffle(idx)
    blocks = []
    for i in range(0, n_points, npts):
        b = idx[i:i+npts]
        if len(b) < npts:
            b = np.pad(b, (0, npts-len(b)), mode="wrap")
        blocks.append(b)
    return blocks

raw = laspy.read(RAW_FILE)
X_all = build_features(raw)
X_all = fit_C(X_all, C_in)

N = X_all.shape[0]
NPTS = 4096
blocks = make_blocks(N, NPTS)

pred = np.zeros(N, dtype=np.int32)

print("Predicting blocks:", len(blocks))
with torch.no_grad():
    for b in tqdm(blocks):
        xb = X_all[b].copy()

        # simple per-block normalization
        xb[:, 0] -= xb[:, 0].mean()
        xb[:, 1] -= xb[:, 1].mean()
        xb[:, 2] -= xb[:, 2].min()

        xt = torch.from_numpy(xb).float().unsqueeze(0).to(DEVICE)  # [1,NPTS,C]
        logits = model(xt)                                         # [1,NPTS,num_classes]
        pb = logits.argmax(-1).squeeze(0).cpu().numpy().astype(np.int32)
        pred[b] = pb

# Save predicted LAZ
out = laspy.LasData(header=raw.header)
out.points = raw.points
out.classification = pred.astype(np.uint8)
out.write(PRED_FILE)
print("✅ Saved:", PRED_FILE)

# -----------------------------
# 5) REAL ACCURACY (GT vs PRED) — SAME POINTS = TRUE ACCURACY
# -----------------------------
gt = laspy.read(GT_FILE)
y_true = np.array(gt.classification, dtype=np.int32)
y_pred = pred.astype(np.int32)

# report only main classes (edit if you want more)
labels = [1, 2, 3, 6]
mask = np.isin(y_true, labels)

rep = classification_report(y_true[mask], y_pred[mask], labels=labels, digits=4, zero_division=0, output_dict=True)
cm  = confusion_matrix(y_true[mask], y_pred[mask], labels=labels)
overall = (np.trace(cm) / np.sum(cm)) * 100.0 if cm.sum() > 0 else 0.0

print("\nREAL ACCURACY (PointNet)\n")
for c in [2, 3, 6]:
    print(f"Class {c} Accuracy (Recall): {rep[str(c)]['recall']*100:.2f}%")
print(f"\nOverall Accuracy: {overall:.2f}%")

Device: cuda
GPU: NVIDIA GeForce RTX 3050
Checkpoint num_classes: 5
✅ Loaded PointNet weights.
Model expects input features C = 10
Predicting blocks: 430


100%|██████████| 430/430 [00:00<00:00, 500.01it/s]


✅ Saved: d:\lidarrrrr\anbu\test\RAW_000005_PRED_PT.laz

REAL ACCURACY (PointNet)

Class 2 Accuracy (Recall): 99.96%
Class 3 Accuracy (Recall): 0.00%
Class 6 Accuracy (Recall): 0.00%

Overall Accuracy: 35.63%


In [7]:
import laspy
import numpy as np

gt = laspy.read(r"D:\lidarrrrr\anbu\OUTPUT FILE\DX3013595 PASQUILIO\LAZ\DX3013595 PASQUILIO000001.laz" )

cls = np.array(gt.classification)

print("Unique GT Classes:")
print(np.unique(cls))

Unique GT Classes:
[ 0  1  2  3  4  5 12 14 16 17 19 21 22]


In [32]:
import laspy
import numpy as np

gt = laspy.read(r"D:\lidarrrrr\anbu\OUTPUT FILE\DX3013595 PASQUILIO\LAZ\DX3013595 PASQUILIO000001.laz" )

cls = np.array(gt.classification)

# ASPRS → MODEL REMAP
remap = {
    0:1,
    1:1,
    2:2,
    3:3,
    4:3,
    5:6,
    12:1,
    14:1,
    16:1,
    17:1,
    19:3,
    21:3,
    22:3
}

for k,v in remap.items():
    cls[cls==k] = v

gt.classification = cls
gt.write(r"d:\lidarrrrr\anbu\test\GT_000001_MODEL.laz")

print("After remap:",np.unique(cls))

After remap: [1 2 3 6]


In [33]:
raw = laspy.read(r"d:\lidarrrrr\anbu\test\GT_000001_MODEL.laz")
raw.classification[:] = 1
raw.write(r"d:\lidarrrrr\anbu\test\RAW_000001.laz")

In [34]:
gt   = laspy.read(r"d:\lidarrrrr\anbu\test\GT_000001_MODEL.laz")
pred = laspy.read(r"d:\lidarrrrr\anbu\test\RAW_000001_PRED_PT.laz")

from sklearn.metrics import classification_report,confusion_matrix
import numpy as np

y_true = np.array(gt.classification)
y_pred = np.array(pred.classification)

labels = [1,2,3,6]

mask = np.isin(y_true,labels)

report = classification_report(
    y_true[mask],
    y_pred[mask],
    labels=labels,
    digits=4,
    zero_division=0,
    output_dict=True
)

cm = confusion_matrix(y_true[mask],y_pred[mask],labels=labels)

overall = (np.trace(cm)/np.sum(cm))*100

print("\nREAL ACCURACY\n")

print(f"Ground (2): {report['2']['recall']*100:.2f}%")
print(f"Vegetation (3): {report['3']['recall']*100:.2f}%")
print(f"Building (6): {report['6']['recall']*100:.2f}%")

print(f"\nOverall: {overall:.2f}%")


REAL ACCURACY

Ground (2): 99.99%
Vegetation (3): 0.00%
Building (6): 0.00%

Overall: 12.26%


In [35]:
import laspy
import numpy as np
from sklearn.metrics import classification_report,confusion_matrix

gt   = laspy.read(r"d:\lidarrrrr\anbu\test\GT_000001_MODEL.laz")
pred = laspy.read(r"d:\lidarrrrr\anbu\test\RAW_000001_PRED_PT.laz")

y_true = np.array(gt.classification)
y_pred = np.array(pred.classification)

labels = np.unique(y_true)   # auto detect GT classes

mask = np.isin(y_true,labels)

report = classification_report(
    y_true[mask],
    y_pred[mask],
    labels=labels,
    digits=4,
    zero_division=0,
    output_dict=True
)

cm = confusion_matrix(y_true[mask],y_pred[mask],labels=labels)

overall = (np.trace(cm)/np.sum(cm))*100

print("\nREAL ACCURACY\n")

for c in labels:
    if str(c) in report:
        print(f"Class {c} Accuracy: {report[str(c)]['recall']*100:.2f}%")

print(f"\nOverall Accuracy: {overall:.2f}%")


REAL ACCURACY

Class 1 Accuracy: 0.00%
Class 2 Accuracy: 99.99%
Class 3 Accuracy: 0.00%
Class 6 Accuracy: 0.00%

Overall Accuracy: 12.26%


In [36]:
import laspy
import numpy as np

gt = laspy.read(r"D:\lidarrrrr\anbu\OUTPUT FILE\DX3013595 PASQUILIO\LAZ\DX3013595 PASQUILIO000002.laz" )

cls = np.array(gt.classification)

# ASPRS → MODEL REMAP
remap = {
    0:1,
    1:1,
    2:2,
    3:3,
    4:3,
    5:6,
    12:1,
    14:1,
    16:1,
    17:1,
    19:3,
    21:3,
    22:3
}

for k,v in remap.items():
    cls[cls==k] = v

gt.classification = cls
gt.write(r"d:\lidarrrrr\anbu\test\GT_000002_MODEL.laz")

print("After remap:",np.unique(cls))

After remap: [1 2 3 6]


In [37]:
raw = laspy.read(r"d:\lidarrrrr\anbu\test\GT_000002_MODEL.laz")
raw.classification[:] = 1
raw.write(r"d:\lidarrrrr\anbu\test\RAW_000002.laz")

In [38]:
gt   = laspy.read(r"d:\lidarrrrr\anbu\test\GT_000002_MODEL.laz")
pred = laspy.read(r"d:\lidarrrrr\anbu\test\RAW_000002_PRED_PT.laz")

from sklearn.metrics import classification_report,confusion_matrix
import numpy as np

y_true = np.array(gt.classification)
y_pred = np.array(pred.classification)

labels = [1,2,3,6]

mask = np.isin(y_true,labels)

report = classification_report(
    y_true[mask],
    y_pred[mask],
    labels=labels,
    digits=4,
    zero_division=0,
    output_dict=True
)

cm = confusion_matrix(y_true[mask],y_pred[mask],labels=labels)

overall = (np.trace(cm)/np.sum(cm))*100

print("\nREAL ACCURACY\n")

print(f"Ground (2): {report['2']['recall']*100:.2f}%")
print(f"Vegetation (3): {report['3']['recall']*100:.2f}%")
print(f"Building (6): {report['6']['recall']*100:.2f}%")

print(f"\nOverall: {overall:.2f}%")


REAL ACCURACY

Ground (2): 99.99%
Vegetation (3): 0.00%
Building (6): 0.00%

Overall: 9.12%


In [39]:
import laspy
import numpy as np
from sklearn.metrics import classification_report,confusion_matrix

gt   = laspy.read(r"d:\lidarrrrr\anbu\test\GT_000002_MODEL.laz")
pred = laspy.read(r"d:\lidarrrrr\anbu\test\RAW_000002_PRED_PT.laz")

y_true = np.array(gt.classification)
y_pred = np.array(pred.classification)

labels = np.unique(y_true)   # auto detect GT classes

mask = np.isin(y_true,labels)

report = classification_report(
    y_true[mask],
    y_pred[mask],
    labels=labels,
    digits=4,
    zero_division=0,
    output_dict=True
)

cm = confusion_matrix(y_true[mask],y_pred[mask],labels=labels)

overall = (np.trace(cm)/np.sum(cm))*100

print("\nREAL ACCURACY\n")

for c in labels:
    if str(c) in report:
        print(f"Class {c} Accuracy: {report[str(c)]['recall']*100:.2f}%")

print(f"\nOverall Accuracy: {overall:.2f}%")


REAL ACCURACY

Class 1 Accuracy: 0.00%
Class 2 Accuracy: 99.99%
Class 3 Accuracy: 0.00%
Class 6 Accuracy: 0.00%

Overall Accuracy: 9.12%


In [40]:
import laspy
import numpy as np

gt = laspy.read(r"D:\lidarrrrr\anbu\OUTPUT FILE\DX3013595 PASQUILIO\LAZ\DX3013595 PASQUILIO000003.laz" )

cls = np.array(gt.classification)

# ASPRS → MODEL REMAP
remap = {
    0:1,
    1:1,
    2:2,
    3:3,
    4:3,
    5:6,
    12:1,
    14:1,
    16:1,
    17:1,
    19:3,
    21:3,
    22:3
}

for k,v in remap.items():
    cls[cls==k] = v

gt.classification = cls
gt.write(r"d:\lidarrrrr\anbu\test\GT_000003_MODEL.laz")

print("After remap:",np.unique(cls))

After remap: [1 2 3 6]


In [41]:
raw = laspy.read(r"d:\lidarrrrr\anbu\test\GT_000003_MODEL.laz")
raw.classification[:] = 1
raw.write(r"d:\lidarrrrr\anbu\test\RAW_000003.laz")

In [42]:
import laspy
import numpy as np
from sklearn.metrics import classification_report,confusion_matrix

gt   = laspy.read(r"d:\lidarrrrr\anbu\test\GT_000003_MODEL.laz")
pred = laspy.read(r"d:\lidarrrrr\anbu\test\RAW_000003_PRED_PT.laz")

y_true = np.array(gt.classification)
y_pred = np.array(pred.classification)

labels = np.unique(y_true)   # auto detect GT classes

mask = np.isin(y_true,labels)

report = classification_report(
    y_true[mask],
    y_pred[mask],
    labels=labels,
    digits=4,
    zero_division=0,
    output_dict=True
)

cm = confusion_matrix(y_true[mask],y_pred[mask],labels=labels)

overall = (np.trace(cm)/np.sum(cm))*100

print("\nREAL ACCURACY\n")

for c in labels:
    if str(c) in report:
        print(f"Class {c} Accuracy: {report[str(c)]['recall']*100:.2f}%")

print(f"\nOverall Accuracy: {overall:.2f}%")


REAL ACCURACY

Class 1 Accuracy: 0.00%
Class 2 Accuracy: 99.89%
Class 3 Accuracy: 0.03%
Class 6 Accuracy: 0.00%

Overall Accuracy: 14.79%


In [43]:
import laspy
import numpy as np

gt = laspy.read(r"D:\lidarrrrr\anbu\OUTPUT FILE\DX3013595 PASQUILIO\LAZ\DX3013595 PASQUILIO000004.laz" )

cls = np.array(gt.classification)

# ASPRS → MODEL REMAP
remap = {
    0:1,
    1:1,
    2:2,
    3:3,
    4:3,
    5:6,
    12:1,
    14:1,
    16:1,
    17:1,
    19:3,
    21:3,
    22:3
}

for k,v in remap.items():
    cls[cls==k] = v

gt.classification = cls
gt.write(r"d:\lidarrrrr\anbu\test\GT_000004_MODEL.laz")

print("After remap:",np.unique(cls))

After remap: [1 2 3 6]


In [44]:
raw = laspy.read(r"d:\lidarrrrr\anbu\test\GT_000004_MODEL.laz")
raw.classification[:] = 1
raw.write(r"d:\lidarrrrr\anbu\test\RAW_000004.laz")

In [45]:
import laspy
import numpy as np
from sklearn.metrics import classification_report,confusion_matrix

gt   = laspy.read(r"d:\lidarrrrr\anbu\test\GT_000004_MODEL.laz")
pred = laspy.read(r"d:\lidarrrrr\anbu\test\RAW_000004_PRED_PT.laz")

y_true = np.array(gt.classification)
y_pred = np.array(pred.classification)

labels = np.unique(y_true)   # auto detect GT classes

mask = np.isin(y_true,labels)

report = classification_report(
    y_true[mask],
    y_pred[mask],
    labels=labels,
    digits=4,
    zero_division=0,
    output_dict=True
)

cm = confusion_matrix(y_true[mask],y_pred[mask],labels=labels)

overall = (np.trace(cm)/np.sum(cm))*100

print("\nREAL ACCURACY\n")

for c in labels:
    if str(c) in report:
        print(f"Class {c} Accuracy: {report[str(c)]['recall']*100:.2f}%")

print(f"\nOverall Accuracy: {overall:.2f}%")


REAL ACCURACY

Class 1 Accuracy: 0.00%
Class 2 Accuracy: 100.00%
Class 3 Accuracy: 0.03%
Class 6 Accuracy: 0.00%

Overall Accuracy: 13.30%


In [46]:
import laspy
import numpy as np

gt = laspy.read(r"D:\lidarrrrr\anbu\OUTPUT FILE\DX3013595 PASQUILIO\LAZ\DX3013595 PASQUILIO000005.laz" )

cls = np.array(gt.classification)

# ASPRS → MODEL REMAP
remap = {
    0:1,
    1:1,
    2:2,
    3:3,
    4:3,
    5:6,
    12:1,
    14:1,
    16:1,
    17:1,
    19:3,
    21:3,
    22:3
}

for k,v in remap.items():
    cls[cls==k] = v

gt.classification = cls
gt.write(r"d:\lidarrrrr\anbu\test\GT_000005_MODEL.laz")

print("After remap:",np.unique(cls))

After remap: [1 2 3]


In [47]:
raw = laspy.read(r"d:\lidarrrrr\anbu\test\GT_000005_MODEL.laz")
raw.classification[:] = 1
raw.write(r"d:\lidarrrrr\anbu\test\RAW_000005.laz")

In [48]:
import laspy
import numpy as np
from sklearn.metrics import classification_report,confusion_matrix

gt   = laspy.read(r"d:\lidarrrrr\anbu\test\GT_000005_MODEL.laz")
pred = laspy.read(r"d:\lidarrrrr\anbu\test\RAW_000005_PRED_PT.laz")

y_true = np.array(gt.classification)
y_pred = np.array(pred.classification)

labels = np.unique(y_true)   # auto detect GT classes

mask = np.isin(y_true,labels)

report = classification_report(
    y_true[mask],
    y_pred[mask],
    labels=labels,
    digits=4,
    zero_division=0,
    output_dict=True
)

cm = confusion_matrix(y_true[mask],y_pred[mask],labels=labels)

overall = (np.trace(cm)/np.sum(cm))*100

print("\nREAL ACCURACY\n")

for c in labels:
    if str(c) in report:
        print(f"Class {c} Accuracy: {report[str(c)]['recall']*100:.2f}%")

print(f"\nOverall Accuracy: {overall:.2f}%")


REAL ACCURACY

Class 1 Accuracy: 0.01%
Class 2 Accuracy: 99.96%
Class 3 Accuracy: 0.00%

Overall Accuracy: 13.43%
